In [1]:
%load_ext lab_black
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import glob
import tqdm
import umap
import lightgbm as lgb
import numpy as np

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

from utils import DATA_SETS, gather_df, prepare_df, model_function

In [ ]:
import optuna
import json

info_file_name = "./sim-res/info.json"
if os.path.exists(info_file_name):
    with open(info_file_name) as f:
        info = json.loads(f.read())
else:
    info = []

for dataset in DATA_SETS:
    df = gather_df(dataset, True)
    target_cols = [c for c in df.columns if "target" in c]

    for target_col in target_cols:
        for pred_ahead in [14, 28, 56]:
            # compute the size of location_matrix_W, which is variable per dataset
            n_feats = (
                len([c for c in df.columns if "ws10m_max" in c])
                + len([c for c in df.columns if "rainfall" in c])
                + len([c for c in df.columns if "temperature" in c])
            )

            # this is the optimization function that we are going to hyperoptimize
            def objective(trial, return_dataframes=False):
                location_array_W = []
                for i in range(n_feats):
                    location_array_W.append(trial.suggest_float("W_{}".format(i), 0, 1))
                dfp_val, dfp_test = model_function(
                    dataset,
                    location_array_W,
                    pred_ahead,
                    target_col,
                    extended_data=trial.suggest_categorical(
                        "extended_data", [True, False]
                    ),
                    impute_missing=trial.suggest_categorical(
                        "impute_missing", [True, False]
                    ),
                    do_extract=trial.suggest_categorical("do_extract", [True, False]),
                    shift_features=trial.suggest_categorical(
                        "shift_features", [True, False]
                    ),
                    use_early_stopping=trial.suggest_categorical(
                        "use_early_stopping", [True, False]
                    ),
                    lgb_boosting_type=trial.suggest_categorical(
                        "lgb_boosting_type", ["gbdt", "goss", "dart"]
                    ),
                    lgb_num_leaves=trial.suggest_categorical(
                        "lgb_num_leaves", [10, 31, 50]
                    ),
                    lgb_learning_rate=trial.suggest_uniform(
                        "lgb_learning_rate", 0.01, 0.3
                    ),
                    lgb_max_depth=trial.suggest_int(
                        "lgb_max_depth", -1, 10
                    ),  # -1 here means infinite
                )
                if return_dataframes:
                    return dfp_val, dfp_test
                return np.mean(
                    np.abs(dfp_val.p - dfp_val.y)
                )  # we optimize on the MAE of the validation dataset

            study = optuna.create_study()
            study.optimize(objective, n_trials=100)

            # the outputting dataframe with optimal parameters
            location_array_W_optim = []
            for k, v in study.best_params.items():
                if "W_" in k:
                    location_array_W_optim.append(v)

            dfp_val, dfp_test = model_function(
                dataset,
                location_array_W_optim,
                pred_ahead,
                target_col,
                extended_data=study.best_params["extended_data"],
                impute_missing=study.best_params["impute_missing"],
                do_extract=study.best_params["do_extract"],
                shift_features=study.best_params["shift_features"],
                use_early_stopping=study.best_params["use_early_stopping"],
                lgb_boosting_type=study.best_params["lgb_boosting_type"],
                lgb_num_leaves=study.best_params["lgb_num_leaves"],
                lgb_learning_rate=study.best_params["lgb_learning_rate"],
                lgb_max_depth=study.best_params["lgb_max_depth"],
            )

            run_id = np.random.randint(1, 100000000000000)

            mae_test = np.mean(np.abs(dfp_test.p - dfp_test.y))
            rmse_test = np.sqrt(np.mean((dfp_test.p - dfp_test.y) ** 2))

            mae_val = np.mean(np.abs(dfp_val.p - dfp_val.y))
            rmse_val = np.sqrt(np.mean((dfp_val.p - dfp_val.y) ** 2))

            info.append(
                {
                    "run_id": run_id,
                    "best_params": study.best_params,
                    "location_weights": location_array_W_optim,
                    "dataset": dataset,
                    "target_col": target_col,
                    "pred_ahead": pred_ahead,
                    "mae_test": mae_test,
                    "rmse_test": rmse_test,
                    "mae_val": mae_val,
                    "rmse_val": rmse_val,
                    "dfp_test_start": str(dfp_test.index[0]),
                    "dfp_test_length": len(dfp_test.index),
                }
            )
          
            with open(info_file_name, "w") as f:
                json.dump(info, f)
            dfp_val.reset_index().to_feather(
                "./sim-res/{}-validation.feather".format(run_id)
            )
            dfp_test.reset_index().to_feather(
                "./sim-res/{}-test.feather".format(run_id)
            )

[I 2021-02-17 09:50:05,645] Trial 0 finished with value: 0.07985704742118081 and parameters: {'W_0': 0.9703649891312696, 'W_1': 0.24989526101978887, 'W_2': 0.5606207978655369, 'W_3': 0.44425861070293005, 'W_4': 0.5492620888530232, 'W_5': 0.8312665176142838, 'W_6': 0.012806917447143018, 'W_7': 0.40310490617511363, 'W_8': 0.3171060638698161, 'W_9': 0.6475096927755477, 'W_10': 0.6478172805866946, 'W_11': 0.9922497328304043, 'W_12': 0.5728973145360226, 'W_13': 0.3401338885801086, 'W_14': 0.033402856459117, 'W_15': 0.4332256062630545, 'W_16': 0.5777727956371453, 'W_17': 0.726178202267492, 'W_18': 0.9158829929366724, 'W_19': 0.9320582766300034, 'W_20': 0.4019425473352348, 'W_21': 0.5600448164326768, 'W_22': 0.37539570699878866, 'W_23': 0.9723638807006302, 'W_24': 0.698594967319482, 'W_25': 0.708776714674995, 'extended_data': False, 'impute_missing': False, 'do_extract': True, 'shift_features': False, 'use_early_stopping': False, 'lgb_boosting_type': 'gbdt', 'lgb_num_leaves': 31, 'lgb_learnin

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 09:50:07,167] Trial 3 finished with value: 0.07147600937340531 and parameters: {'W_0': 0.7059884695078009, 'W_1': 0.2928432025669365, 'W_2': 0.3016352237050578, 'W_3': 0.5909952338510811, 'W_4': 0.4145386986548092, 'W_5': 0.9392404766165543, 'W_6': 0.4551360973987295, 'W_7': 0.9235309770784632, 'W_8': 0.10863535246603828, 'W_9': 0.590290527388406, 'W_10': 0.7218317076085806, 'W_11': 0.9412950318281682, 'W_12': 0.2716321748148085, 'W_13': 0.7149968621357236, 'W_14': 0.1410566124249446, 'W_15': 0.397768887769959, 'W_16': 0.5496672165088556, 'W_17': 0.25454877904653606, 'W_18': 0.4647354220522567, 'W_19': 0.8154043016712771, 'W_20': 0.23491881822528293, 'W_21': 0.9292716393673809, 'W_22': 0.19847424365113053, 'W_23': 0.7269902930047416, 'W_24': 0.7739114135350854, 'W_25': 0.18707209090424404, 'extended_data': True, 'impute_missing': False, 'do_extract': False, 'shift_features': False, 'use_early_stopping': True, 'lgb_boosting_type': 'goss', 'lgb_num_leaves': 31, 'lgb_learnin

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 09:50:08,847] Trial 6 finished with value: 0.07787379097525993 and parameters: {'W_0': 0.43221720648230877, 'W_1': 0.1853275417528919, 'W_2': 0.8008296638786616, 'W_3': 0.13364664498013468, 'W_4': 0.18642927059471037, 'W_5': 0.01510616817605892, 'W_6': 0.9809852072328359, 'W_7': 0.005785441925962198, 'W_8': 0.952913803945791, 'W_9': 0.08045675494791305, 'W_10': 0.41062685726776316, 'W_11': 0.6559158511494795, 'W_12': 0.21658798330046036, 'W_13': 0.09456333858012189, 'W_14': 0.0009508522756022098, 'W_15': 0.28163594170825, 'W_16': 0.9409673381168846, 'W_17': 0.8410284650777743

...

[I 2021-02-17 09:59:21,654] Trial 22 finished with value: 0.02434512259144635 and parameters: {'W_0': 0.18380719215006805, 'W_1': 0.7468772776825583, 'W_2': 0.7081567805258165, 'W_3': 0.1419903047100035, 'W_4': 0.41204794762180785, 'W_5': 0.6191960554386262, 'W_6': 0.10162766570463067, 'W_7': 0.7370938556030182, 'W_8': 0.23140250164272141, 'W_9': 0.8233744764966201, 'W_10': 0.2699427626955756, 'W_11': 0.20698350527244752, 'W_12': 0.5363936309416938, 'extended_data': False, 'impute_missing': False, 'do_extract': True, 'shift_features': True, 'use_early_stopping': True, 'lgb_boosting_type': 'gbdt', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.11036614279375809, 'lgb_max_depth': 6}. Best is trial 5 with value: 0.023931071221659928.


[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 09:59:21,972] Trial 23 finished with value: 0.023771448788459434 and parameters: {'W_0': 0.12183997697222554, 'W_1': 0.7315145312138766, 'W_2': 0.6815909283705518, 'W_3': 0.13951343845839026, 'W_4': 0.5544004233601221, 'W_5': 0.6362169047692522, 'W_6': 0.3291201617645499, 'W_7': 0.535920045759624, 'W_8': 0.1036422215655012, 'W_9': 0.8473599336959894, 'W_10': 0.17000956351028385, 'W_11': 0.19408488526583312, 'W_12': 0.5487418936758565, 'extended_data': False, 'impute_missing': False, 'do_extract': True, 'shift_features': True, 'use_early_stopping': True, 'lgb_boosting_type': 'gbdt', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.011334290755402432, 'lgb_max_depth': 7}. Best is trial 23 with value: 0.023771448788459434.


[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 09:59:22,356] Trial 24 finished with value: 0.024631696876533445 and parameters: {'W_0': 0.5576538053113643, 'W_1': 0.5510355755255932, 'W_2': 0.8523993054761293, 'W_3': 0.00626914498162906, 'W_4': 0.5923513733596656, 'W_5': 0.7297649289251114, 'W_6': 0.3332991879609211, 'W_7': 0.524013060599039, 'W_8': 0.10641898729976562, 'W_9': 0.851513256922909, 'W_10': 0.15262460151849716, 'W_11': 0.0006138589771634145, 'W_12': 0.7667798391521949, 'extended_data': False, 'impute_missing': False, 'do_extract': True, 'shift_features': True, 'use_early_stopping': True, 'lgb_boosting_type': 'gbdt', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.05610393664949369, 'lgb_max_depth': 8}. Best is trial 23 with value: 0.023771448788459434.
[I 2021-02-17 09:59:23,179] Trial 25 finished with value: 0.022901734526804343 and parameters: {'W_0': 0.10025217471597472, 'W_1': 0.7377035806015728, 'W_2': 0.7168959482763907, 'W_3': 0.1130714090620605, 'W_4': 0.7074443215975078, 'W_5': 0.7979349185433149, '

In [42]:
dfp_val.reset_index().to_feather("./sim-res/{}-validation.feather".format(run_id))
dfp_test.reset_index().to_feather("./sim-res/{}-test.feather".format(run_id))

# info.append({
#     'run_id': run_id,
#     'best_params': best_params,
#     'location_weights': location_array_W_optim,
#     'dataset': dataset,
#     'target_col': target_col,
#     'pred_ahead': pred_ahead
# })
# with open(info_file_name, 'w') as f:
#     json.dump(info, f)